In [2]:
import warnings

# Ignorer tous les avertissements
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [4]:
df = pd.read_excel('sensor_readings_4.xlsx')

df.head()

,S0_front,S0_left,S0_right,S0_back,class
0,1.687,0.445,2.332,0.429,Slight-Right-Turn
1,1.687,0.449,2.332,0.429,Slight-Right-Turn
2,1.687,0.449,2.334,0.429,Slight-Right-Turn
3,1.687,0.449,2.334,0.429,Slight-Right-Turn
4,1.687,0.449,2.334,0.429,Slight-Right-Turn


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# --- 1. Chargement des données ---
# Assurez-vous que le fichier est dans le même dossier ou indiquez le chemin complet
# df = pd.read_csv('sensor_readings_4.csv') # ou pd.read_excel(...) selon votre fichier
# Pour l'exemple, je reprends votre variable df existante :
# df = ...

# --- 2. Préparation des Entrées (X) et Sorties (y) ---

# Sélection des 4 premières colonnes comme features (capteurs)
X = df.iloc[:, :-1].values

# Sélection de la dernière colonne comme target (classe)
y_raw = df.iloc[:, -1].values

# --- 3. Encodage de la cible (One-Hot Encoding) ---
# Transformation des textes (ex: "Slight-Right-Turn") en vecteurs (ex: [0, 0, 0, 1])
# Cela crée automatiquement 4 colonnes correspondant aux 4 sorties demandées
encoder = LabelBinarizer()
y = encoder.fit_transform(y_raw)

# On affiche les classes pour vérifier l'ordre (ex: Move-Forward, Sharp-Right, etc.)
print("Classes encodées :", encoder.classes_)

# --- 4. Création des ensembles d'entraînement et de test ---
# On divise les données (par exemple 70% entraînement, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vérification des dimensions
print(f"Dimensions X_train: {X_train.shape}") # Devrait être (N, 4)
print(f"Dimensions y_train: {y_train.shape}") # Devrait être (N, 4)

Classes encodées : ['Move-Forward' 'Sharp-Right-Turn' 'Slight-Left-Turn' 'Slight-Right-Turn']
Dimensions X_train: (3819, 4)
Dimensions y_train: (3819, 4)


In [6]:
import pandas as pd
import numpy as np
import itertools
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

# --- Configuration ---
possible_neurons = [4, 8, 10, 15, 20]
max_layers = 4
EPOCHS = 50
BATCH_SIZE = 32
REPETITIONS = 3

# Génération des architectures (1 à 4 couches, décroissantes ou égales)
architectures = []
# 1 couche
for n in possible_neurons:
    architectures.append([n])
# 2 à 4 couches
for k in range(2, max_layers + 1):
    for combo in itertools.combinations_with_replacement(possible_neurons, k):
        arch = sorted(list(combo), reverse=True) # Triangulaire
        architectures.append(arch)

print(f"Configurations à tester : {len(architectures)}")
print(f"Total des entraînements : {len(architectures) * REPETITIONS}")


Configurations à tester : 125
Total des entraînements : 375


In [ ]:

# Base de données pour stocker TOUS les résultats
results_db = []

print("Lancement des entraînements...")

for arch_idx, arch in enumerate(architectures):
    arch_name = str(arch)
    print(f"\nConfiguration {arch_idx+1}/{len(architectures)} : {arch_name}")

    for run_id in range(REPETITIONS):
        # Construction du modèle
        model = keras.Sequential()
        model.add(layers.InputLayer(input_shape=(4,)))
        for n in arch:
            model.add(layers.Dense(n, activation='relu'))
        model.add(layers.Dense(4, activation='softmax'))

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Callback
        es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=0)

        # Entraînement
        history = model.fit(X_train, y_train,
                            epochs=EPOCHS,
                            batch_size=BATCH_SIZE,
                            validation_split=0.2,
                            verbose=0,
                            callbacks=[es])

        # Prédictions et Évaluation
        y_pred_prob = model.predict(X_test, verbose=0)
        y_pred_classes = np.argmax(y_pred_prob, axis=1)
        y_true_classes = np.argmax(y_test, axis=1)
        acc = accuracy_score(y_true_classes, y_pred_classes)

        # Stockage complet
        entry = {
            'arch_id': arch_idx,
            'arch_name': arch_name,
            'num_layers': len(arch),
            'complexity': model.count_params(),
            'run_id': run_id,
            'accuracy': acc,
            'history': history.history, # Sauvegarde des courbes
            'y_pred_classes': y_pred_classes, # Pour la matrice de confusion future
            'model': model # (Optionnel, attention à la RAM si gros volume)
        }
        results_db.append(entry)
        print(f"   Run {run_id+1}: Acc={acc:.4f} | Epochs={len(history.history['loss'])}")

# Création d'un DataFrame résumé pour l'analyse facile
df_summary = pd.DataFrame(results_db).drop(columns=['history', 'y_pred_classes', 'model'])
print("\nTerminé ! Données prêtes pour analyse.")

Lancement des entraînements...

Configuration 1/125 : [4]
   Run 1: Acc=0.7697 | Epochs=50
   Run 2: Acc=0.7911 | Epochs=50
   Run 3: Acc=0.7324 | Epochs=50

Configuration 2/125 : [8]
   Run 1: Acc=0.8888 | Epochs=50
   Run 2: Acc=0.8900 | Epochs=50
   Run 3: Acc=0.8833 | Epochs=50

Configuration 3/125 : [10]
   Run 1: Acc=0.8937 | Epochs=50
   Run 2: Acc=0.8644 | Epochs=50
   Run 3: Acc=0.8919 | Epochs=50

Configuration 4/125 : [15]
   Run 1: Acc=0.9230 | Epochs=50
   Run 2: Acc=0.9126 | Epochs=50
   Run 3: Acc=0.8992 | Epochs=50

Configuration 5/125 : [20]
   Run 1: Acc=0.9267 | Epochs=50
   Run 2: Acc=0.9181 | Epochs=50
   Run 3: Acc=0.9071 | Epochs=50

Configuration 6/125 : [4, 4]
   Run 1: Acc=0.8693 | Epochs=50
   Run 2: Acc=0.8173 | Epochs=50
   Run 3: Acc=0.7392 | Epochs=50

Configuration 7/125 : [8, 4]
   Run 1: Acc=0.8247 | Epochs=50
   Run 2: Acc=0.9224 | Epochs=50
   Run 3: Acc=0.8656 | Epochs=50

Configuration 8/125 : [10, 4]
   Run 1: Acc=0.8228 | Epochs=50
   Run 2: Acc=

In [ ]:
import plotly.express as px

# On trie par accuracy médiane pour avoir les meilleurs à gauche/droite
df_sorted = df_summary.sort_values(by='accuracy', ascending=True)

fig_stability = px.box(df_sorted,
                       x='arch_name',
                       y='accuracy',
                       points="all", # Affiche tous les points (les 3 runs)
                       hover_data=['complexity', 'run_id'],
                       title="Stabilité et Performance par Architecture (3 runs chacune)",
                       labels={'arch_name': 'Architecture (Neurones)', 'accuracy': 'Précision (Test)'},
                       color='num_layers') # Couleur par nombre de couches

fig_stability.update_layout(xaxis_tickangle=-45)
fig_stability.show()

In [ ]:
import plotly.graph_objects as go

# Création de la figure
fig_loss = go.Figure()

# Liste des architectures uniques
unique_archs = df_summary['arch_name'].unique()

# On ajoute TOUTES les traces, mais on les rend invisibles par défaut sauf la première
first_arch = unique_archs[0]

for arch in unique_archs:
    # Récupérer les 3 runs de cette arch
    runs = [res for res in results_db if res['arch_name'] == arch]

    for i, run in enumerate(runs):
        hist = run['history']
        epochs = list(range(1, len(hist['loss']) + 1))

        # Trace Loss Train
        fig_loss.add_trace(go.Scatter(
            x=epochs, y=hist['loss'],
            mode='lines',
            name=f'Run {i+1} Train',
            line=dict(width=2),
            visible=(arch == first_arch), # Visible seulement si c'est la 1ere arch
            legendgroup=f'group{arch}'
        ))

        # Trace Loss Val (pointillés)
        fig_loss.add_trace(go.Scatter(
            x=epochs, y=hist['val_loss'],
            mode='lines',
            name=f'Run {i+1} Val',
            line=dict(dash='dash', width=2),
            visible=(arch == first_arch),
            legendgroup=f'group{arch}'
        ))

# Création du menu déroulant
buttons = []
traces_per_arch = REPETITIONS * 2 # 2 traces (Train/Val) par run

for i, arch in enumerate(unique_archs):
    # Création du masque de visibilité : [Faux, Faux, ..., Vrai, Vrai, ..., Faux]
    visible = [False] * (len(unique_archs) * traces_per_arch)
    start_idx = i * traces_per_arch
    end_idx = start_idx + traces_per_arch
    visible[start_idx:end_idx] = [True] * traces_per_arch

    buttons.append(dict(
        label=arch,
        method="update",
        args=[{"visible": visible},
              {"title": f"Courbes de perte pour l'architecture : {arch}"}]
    ))

fig_loss.update_layout(
    updatemenus=[dict(active=0, buttons=buttons, x=1.15, y=1)],
    title=f"Courbes de perte (Loss) - {first_arch}",
    xaxis_title="Époque",
    yaxis_title="Loss (Perte)",
    hovermode="x unified"
)

fig_loss.show()

In [ ]:
# --- SÉLECTION DE L'UTILISATEUR ---
# Copiez ici le nom de l'architecture qui vous intéresse (vu dans le graphe précédent)
CHOIX_ARCHITECTURE = "[20, 10]"  # Exemple, à modifier selon vos résultats

# ----------------------------------
from sklearn.metrics import confusion_matrix, classification_report

# Récupération des runs pour cette architecture
candidates = [res for res in results_db if res['arch_name'] == CHOIX_ARCHITECTURE]

if not candidates:
    print(f"Erreur : L'architecture {CHOIX_ARCHITECTURE} est introuvable. Vérifiez l'orthographe (espaces, crochets).")
else:
    # On choisit le MEILLEUR run parmi les 3 (celui avec la meilleure accuracy)
    best_run = max(candidates, key=lambda x: x['accuracy'])

    print(f"Architecture : {best_run['arch_name']}")
    print(f"Meilleur Run ID : {best_run['run_id']}")
    print(f"Précision : {best_run['accuracy']:.4f}")

    # Matrice de confusion
    cm = confusion_matrix(np.argmax(y_test, axis=1), best_run['y_pred_classes'])
    class_names = encoder.classes_

    fig_cm = px.imshow(cm,
                       text_auto=True,
                       labels=dict(x="Classe Prédite", y="Classe Réelle", color="Nombre"),
                       x=class_names,
                       y=class_names,
                       color_continuous_scale='RdBu_r', # Palette différente
                       title=f"Matrice de Confusion : {CHOIX_ARCHITECTURE} (Run {best_run['run_id']})")
    fig_cm.show()

    # Rapport textuel
    print("\nRapport de Classification :")
    print(classification_report(np.argmax(y_test, axis=1), best_run['y_pred_classes'], target_names=class_names))